Ver arquivo mqo_formula_explanation

In [220]:
from pathlib import Path

import polars as pl


In [221]:

pasta_arquivos = Path("data/raw")

# Ler arquivos
ibov = pl.read_json(pasta_arquivos / "history_ibov.json")
selic = pl.read_json(pasta_arquivos / "history_selic.json")
selic_meta = pl.read_json(pasta_arquivos / "history_selic_meta.json")

# Converter datas
dfs = [ibov, selic, selic_meta]
dfs = [
    df.with_columns(
        pl.col("date_reference").str.to_datetime()
    )
    for df in dfs
]

ibov, selic, selic_meta = dfs

# Renomear colunas
ibov = ibov.rename({"value": "ibov"})
selic = selic.rename({"value": "selic_mensal"})
selic_meta = selic_meta.rename({"value": "selic_meta"})

# Merge (join) em Polars
df = (
    ibov.select(["date_reference", "ibov"])
        .join(
            selic.select(["date_reference", "selic_mensal"]),
            on="date_reference",
            how="inner"
        )
        .join(
            selic_meta.select(["date_reference", "selic_meta"]),
            on="date_reference",
            how="inner"
        )
)

# Remover nulos e resetar index (Polars não precisa resetar index)
df = df.drop_nulls()

df.head(10)
ctx = pl.SQLContext()
ctx.register("df", df)


<SQLContext [tables:1] at 0x26ceb93e0d0>

In [317]:
year_start = "2009"
year_end = "2024"

base = ctx.execute(f"""
    select
        strftime(date_reference, '%Y%m')        as date_ref,
        ibov                                    as Y,
        selic_meta                              as X
    from
        df
    where
        strftime(date_reference, '%Y') between '{year_start}' and '{year_end}'
""").collect()

ctx.register("base", base)

stats = ctx.execute("""
    select
        avg(Y)              as Y_media,     -- Média Y Amostral
        avg(X)              as X_media,     -- Média X Amostral
        count(*)::double    as n
    from
        base
""").collect()

ctx.register("stats", stats)

stats = ctx.execute("""
    select
        max(Y_media)                                    as Y_media,   -- Média Y Amostral
        max(X_media)                                    as X_media,   -- Média X Amostral
        max(n)                                          as n,         -- Quantidade Amostra
        sum((X - X_media)*(Y - Y_media))                as SXY,       -- Covariância Não Normalizada
        sum(pow(X - X_media , 2))                       as SXX,       -- Variância Não Normalizada de X
        sum(pow(Y - Y_media , 2))                       as SQT,       -- Soma Total dos Quadrados
        sum((X - X_media)*(Y - Y_media)) /
        sum(pow(X - X_media , 2))                       as beta_1,    -- Estimador de Inclinação(SXY/SXX)
        max(Y_media) -
            (
                sum((X - X_media)*(Y - Y_media)) /
                sum(pow(X - X_media , 2)) 
            )*max(X_media)                              as beta_0,    -- Estimador de Intercepto (Y_m-B1*X_m)
    from
        base
        cross join stats
""").collect()
ctx.register("stats", stats)

base = ctx.execute("""
    select
        date_ref                            as date_ref,
        Y                                   as Y,
        X                                   as X,
        pow(X , 2)                          as X2,
        beta_0 + beta_1 * X                 as Y_estim,
        pow(Y - (beta_0 + beta_1 * X),2)    as u2_residuo, -- Resíduo ao Quadrado
    from
        base
        cross join stats
""").collect()
ctx.register("base", base)

stats = ctx.execute("""
    select
        max(Y_media)                as Y_media,
        max(X_media)                as X_media,
        max(n)                      as n,
        max(SXY)                    as SXY,
        max(SXX)                    as SXX,
        max(SQT)                    as SQT,
        max(beta_1)                 as beta_1,
        max(beta_0)                 as beta_0,
        sum(X2)                     as SX2,         -- Soma do X^2
        sum(u2_residuo)             as SQE,         -- Soma dos Quadrados dos Erros
        max(SQT) - sum(u2_residuo)  as SQR,         -- Soma dos Quadrados da Regressão(SQT-SQE)
    from
        base
        cross join stats
""").collect()
ctx.register("stats", stats)

stats = ctx.execute("""
    select
        stats.*,
        SQR/SQT                         as R2,              -- O percentual da variação do Y explicado pelo X
        1-(1-(SQR/SQT))*(n-1)/(n-2)     as R2_ajustado,     -- Corrigido por graus de liberdade, mais útil em MRLM.
        SQE/(n-2)                       as var2,            -- Variância do Erro
        (SQE/(n-2))/SXX                 as var_beta_1,      -- Variância do Estimador beta_1_estim
        (SQE/(n-2))*(SX2/(n*SXX))       as var_beta_0,      -- Variância do Estimado beta_0_estim
    from
        stats
""").collect()
ctx.register("stats", stats)

stats = ctx.execute("""
    select
        stats.*,
        (SQR/1) / (SQE/(n-2))           as F,                           -- Testa a significância geral do modelo.
        SQRT(var_beta_1)                as erro_padrao_beta_1,          -- Erros Padrão beta_1 (SE)
        SQRT(var_beta_0)                as erro_padrao_beta_0,          -- Erros Padrão beta_0 (SE)
        beta_1 / SQRT(var_beta_1)       as t_beta_1,                    -- Estatística t para beta_1 (SELIC)
        beta_0 / SQRT(var_beta_0)       as t_beta_0,                    -- Estatística t para beta_0 (Intercepto)
    from
        stats
""").collect()
ctx.register("stats", stats)



<SQLContext [tables:3] at 0x26ceb93e0d0>

In [318]:
ctx.execute("""
    select
        stats.*,
    from
        stats
""").collect()

Y_media,X_media,n,SXY,SXX,SQT,beta_1,beta_0,SX2,SQE,SQR,R2,R2_ajustado,var2,var_beta_1,var_beta_0,F,erro_padrao_beta_1,erro_padrao_beta_0,t_beta_1,t_beta_0
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
79231.814916,9.512656,192.0,-5.0472e6,2187.200145,1.4330e11,-2307.606227,101183.279717,19561.4009,1.3165e11,1.1647e10,0.081278,0.076442,6.9290e8,316798.048149,3.2276e7,16.808962,562.848157,5681.206969,-4.099873,17.810173


In [283]:
R

-0.27745057242864996